# Li(110) Step-edge NEB with LCAO (Low RAM)

This notebook builds a Li bcc (110) slab with a monoatomic step edge and
computes the diffusion barrier for a Li adatom moving from the lower
terrace to the upper terrace across the step using NEB.

It uses **GPAW in LCAO mode** to keep the memory footprint low (target
≈ 4 GB) and avoids writing `.traj` files to sidestep JSON/ULM issues.


## 1. Imports


In [1]:
from ase.build import surface
from ase.io import write, read
from ase import Atom
from ase.optimize import BFGS, FIRE
from ase.mep import NEB
from ase.constraints import FixAtoms
from ase.parallel import world

from gpaw import GPAW, LCAO
from gpaw.occupations import FermiDirac

import numpy as np
import os
import json
import time

print("Imports OK")

Imports OK


## 2. Parameters (LCAO + low RAM)


In [2]:
layers = 6              # number of (110) layers before carving the step
supercell = (4, 4, 1)   # repetition in x, y, z
vacuum = 10.0           # Å of vacuum above the surface
adatom_height = 2.0     # Å above local top terrace

xc = "PBE"
kp = 2                  # k-point sampling (kp x kp x 1)
initial_final_fmax = 0.05  # eV/Å for initial/final relaxations
neb_fmax = 0.10            # eV/Å for NEB convergence
neb_max_steps = 200        # max NEB optimization steps
n_middle_images = 5        # intermediate images (total = 7)

# LCAO-specific controls to keep RAM usage reasonable
lcao_basis = "dzp"          # double-zeta + polarization
fermi_width = 0.15          # eV, smearing for convergence
ram_budget_gb = 4.0         # soft memory target (for your reference)
lcao_mode = LCAO(force_complex_dtype=False)

output_dir = "Li110_step_NEB_results"
os.makedirs(output_dir, exist_ok=True)

# Status file for restart capability
status_path = os.path.join(output_dir, "status.json")

if world.rank == 0:
    print(f"Output directory: {output_dir}")
    print(f"Status file: {status_path}")
    print(f"Using LCAO basis '{lcao_basis}', target ~{ram_budget_gb} GB")

Output directory: Li110_step_NEB_results
Status file: Li110_step_NEB_results/status.json
Using LCAO basis 'dzp', target ~4.0 GB


## 3. Helper functions


In [3]:
def build_li110_step_slab():
    """Build a Li bcc(110) slab with a monoatomic step edge."""
    slab = surface("Li", (1, 1, 0), layers=layers, vacuum=vacuum)
    slab = slab.repeat(supercell)

    pos = slab.get_positions()
    x = pos[:, 0]
    z = pos[:, 2]
    z_max = z.max()

    # Identify top layer atoms
    dz_top = 0.5
    top_indices = np.where(z > z_max - dz_top)[0]

    # Use midpoint of top-layer x's as approximate step location
    x_mid = 0.5 * (x[top_indices].min() + x[top_indices].max())

    # Delete top-layer atoms with x > x_mid to create the step
    to_delete = [i for i in top_indices if x[i] > x_mid]
    for i in sorted(to_delete, reverse=True):
        del slab[i]

    return slab


def find_upper_lower_terrace_sites(slab):
    """Find one site on lower and one on upper terrace near the step.

    Strategy:
    - Cluster z-coordinates into discrete terraces.
    - Identify the two highest terraces (upper and lower).
    - Among all pairs (lower_i, upper_j), choose the pair with the smallest
      lateral (x,y) distance. This should naturally pick atoms near the
      step edge, without assuming any particular x-direction.
    """
    pos = slab.get_positions()
    x = pos[:, 0]
    y = pos[:, 1]
    z = pos[:, 2]

    # Cluster z levels
    z_sorted = np.sort(z)
    levels = []
    tol = 0.3  # Å tolerance to group atoms into the same terrace
    for zi in z_sorted:
        if not levels or abs(zi - levels[-1]) > tol:
            levels.append(zi)

    if len(levels) < 2:
        raise RuntimeError("Could not identify at least two distinct terraces in z.")

    z_hi = levels[-1]
    z_lo = levels[-2]

    # Atoms belonging to upper and lower terraces
    upper = np.where(abs(z - z_hi) < tol)[0]
    lower = np.where(abs(z - z_lo) < tol)[0]

    if len(upper) == 0 or len(lower) == 0:
        raise RuntimeError("Could not find atoms on upper and lower terraces.")

    # Choose the closest pair (lower_i, upper_j) in lateral distance
    best_pair = None
    best_dist = 1e9
    for i in lower:
        for j in upper:
            dx = x[j] - x[i]
            dy = y[j] - y[i]
            d_xy = np.sqrt(dx*dx + dy*dy)
            if d_xy < best_dist and d_xy > 1e-3:
                best_dist = d_xy
                best_pair = (i, j)

    if best_pair is None:
        raise RuntimeError("Could not find a suitable lower/upper terrace pair.")

    i_lower, i_upper = best_pair
    p_lower = pos[i_lower]
    p_upper = pos[i_upper]
    z_max_global = z.max()

    return p_lower, p_upper, z_max_global


def apply_bottom_fix(slab, fraction=0.5):
    """Freeze bottom part of the slab using a z-threshold."""
    z = slab.get_positions()[:, 2]
    z_min, z_max = z.min(), z.max()
    z_cut = z_min + fraction * (z_max - z_min)
    mask = z < z_cut
    slab.set_constraint(FixAtoms(mask=mask))
    return slab


def make_adatom_state(base_slab, site_pos, z_top):
    """Return a new Atoms object with an adatom above a given site."""
    slab = base_slab.copy()
    slab = apply_bottom_fix(slab, fraction=0.5)
    adatom = Atom("Li", [site_pos[0], site_pos[1], z_top + adatom_height])
    slab.append(adatom)
    return slab


def make_gpaw(label):
    """Create a GPAW calculator in LCAO mode with low memory usage."""
    return GPAW(
        mode=lcao_mode,
        basis=lcao_basis,
        xc=xc,
        kpts=(kp, kp, 1),
        occupations=FermiDirac(fermi_width),
        convergence={"eigenstates": 5e-6, "density": 1e-4},
        txt=os.path.join(output_dir, f"{label}.txt"),
    )


# ---------- Status management for restart capability ----------

def load_status():
    """Load status from JSON file, or return empty dict if not found."""
    if not os.path.exists(status_path):
        return {}
    try:
        with open(status_path, "r") as f:
            return json.load(f)
    except Exception:
        return {}


def save_status(status):
    """Save status to JSON file."""
    if world.rank == 0:
        with open(status_path, "w") as f:
            json.dump(status, f, indent=2)


def relax_structure(atoms, label, fmax, allow_restart=True):
    """Relax a structure with BFGS until max force < fmax.

    - Uses a light LCAO GPAW calculator.
    - Writes a `.traj` file to disk for restart capability.
    - If `allow_restart` and `<label>.traj` exists, restart from the
      last saved frame instead of the raw input geometry.
    - Logs step number, max force, and elapsed time.
    """
    traj_path = os.path.join(output_dir, f"{label}.traj")

    if allow_restart and os.path.exists(traj_path):
        if world.rank == 0:
            print(f"[{label}] Restarting from existing trajectory: {traj_path}")
        try:
            atoms = read(traj_path, index=-1)
        except Exception as e:
            if world.rank == 0:
                print(f"[{label}] Could not read existing traj, starting fresh. Reason: {e}")

    atoms.calc = make_gpaw(label)
    dyn = BFGS(
        atoms,
        logfile=os.path.join(output_dir, f"{label}.log"),
        trajectory=traj_path,  # save trajectory for restart
    )

    step = 0
    t0 = time.time()
    if world.rank == 0:
        print(f"[{label}] Starting relaxation (target fmax = {fmax:.3f} eV/Å)")

    for _ in dyn.irun(fmax=fmax):
        step += 1
        if world.rank == 0:
            elapsed = time.time() - t0
            forces = atoms.get_forces()
            maxf = float(np.abs(forces).max())
            print(f"[{label}] step {step:4d}, max |F| = {maxf:.3f} eV/Å, elapsed = {elapsed:7.1f} s")

    if world.rank == 0:
        total = time.time() - t0
        print(f"[{label}] Relaxation finished in {step} steps, total time = {total:.1f} s.")
    return atoms

print("Helper functions defined.")

Helper functions defined.


## 4. Build stepped slab and define initial/final adatom states


In [ ]:
# Build stepped Li(110) slab
slab_step = build_li110_step_slab()
if world.rank == 0:
    write(os.path.join(output_dir, "Li-step-slab.xyz"), slab_step)
    print("Stepped Li(110) slab written to Li-step-slab.xyz")

# Choose lower and upper terrace sites
p_lower, p_upper, z_max = find_upper_lower_terrace_sites(slab_step)
if world.rank == 0:
    print("Lower terrace site:", p_lower)
    print("Upper terrace site:", p_upper)

# Initial: adatom on lower terrace; Final: adatom on upper terrace
initial = make_adatom_state(slab_step, p_lower, z_max)
final = make_adatom_state(slab_step, p_upper, z_max)

if world.rank == 0:
    write(os.path.join(output_dir, "Li-initial-unrelaxed.xyz"), initial)
    write(os.path.join(output_dir, "Li-final-unrelaxed.xyz"),   final)
    print("Unrelaxed initial/final structures written.")

Stepped Li(110) slab written to Li-step-slab.xyz
Lower terrace site: [ 2.46780267  8.725      19.87121067]
Upper terrace site: [ 2.46780267 10.47       22.33901333]
Unrelaxed initial/final structures written.


## 5. Relax initial and final structures


In [ ]:
# Load current status
status = load_status()
if world.rank == 0:
    print("Current status:", status)

# --- Relax initial state (with restart awareness) ---
if not status.get("initial_relaxed", False):
    if world.rank == 0:
        print("Relaxing initial adatom configuration...")
    initial_relaxed = relax_structure(initial, "Li-step-initial", initial_final_fmax, allow_restart=True)
    status["initial_relaxed"] = True
    save_status(status)
    if world.rank == 0:
        write(os.path.join(output_dir, "Li-initial-relaxed.xyz"), initial_relaxed)
        print("Initial relaxation complete. Status saved.")
else:
    if world.rank == 0:
        print("Initial state already relaxed (loading from trajectory)...")
    initial_relaxed = read(os.path.join(output_dir, "Li-step-initial.traj"), index=-1)

# --- Relax final state (with restart awareness) ---
if not status.get("final_relaxed", False):
    if world.rank == 0:
        print("Relaxing final adatom configuration...")
    final_relaxed = relax_structure(final, "Li-step-final", initial_final_fmax, allow_restart=True)
    status["final_relaxed"] = True
    save_status(status)
    if world.rank == 0:
        write(os.path.join(output_dir, "Li-final-relaxed.xyz"), final_relaxed)
        print("Final relaxation complete. Status saved.")
else:
    if world.rank == 0:
        print("Final state already relaxed (loading from trajectory)...")
    final_relaxed = read(os.path.join(output_dir, "Li-step-final.traj"), index=-1)

if world.rank == 0:
    print("\nBoth initial and final states ready for NEB.")

Current status: {}
Relaxing initial adatom configuration...
[Li-step-initial] Restarting from existing trajectory: Li110_step_NEB_results/Li-step-initial.traj
[Li-step-initial] Could not read existing traj, starting fresh. Reason: 
[Li-step-initial] Starting relaxation (target fmax = 0.050 eV/Å)
[Li-step-initial] step    1, max |F| = 0.413 eV/Å, elapsed =  3507.7 s
[Li-step-initial] step    1, max |F| = 0.413 eV/Å, elapsed =  3507.7 s
[Li-step-initial] step    2, max |F| = 0.394 eV/Å, elapsed =  4297.2 s
[Li-step-initial] step    2, max |F| = 0.394 eV/Å, elapsed =  4297.2 s
[Li-step-initial] step    3, max |F| = 0.142 eV/Å, elapsed =  6076.3 s
[Li-step-initial] step    3, max |F| = 0.142 eV/Å, elapsed =  6076.3 s
[Li-step-initial] step    4, max |F| = 0.187 eV/Å, elapsed =  6752.4 s
[Li-step-initial] step    4, max |F| = 0.187 eV/Å, elapsed =  6752.4 s
[Li-step-initial] step    5, max |F| = 0.219 eV/Å, elapsed =  7424.8 s
[Li-step-initial] step    5, max |F| = 0.219 eV/Å, elapsed =  74

## 6. NEB: lower → upper terrace across the step


In [ ]:
# Load current status
status = load_status()

# Check if NEB is already done
if status.get("neb_done", False):
    if world.rank == 0:
        print("NEB already marked as done. Loading results...")
    energies_file = os.path.join(output_dir, "Li-step-NEB-energies.dat")
    if os.path.exists(energies_file):
        data = np.loadtxt(energies_file, comments="#")
        energies = data[:, 1]
        ref = min(energies[0], energies[-1])
        barrier = energies.max() - ref
        if world.rank == 0:
            print(f"Li step-edge diffusion barrier ≈ {barrier:.3f} eV (from saved data)")
    else:
        if world.rank == 0:
            print("NEB marked done but energies file missing. Rerunning NEB.")
        status["neb_done"] = False
        save_status(status)

if not status.get("neb_done", False):
    # Build NEB images
    images = [initial_relaxed]
    images += [initial_relaxed.copy() for _ in range(n_middle_images)]
    images += [final_relaxed]

    neb = NEB(images, climb=True)
    neb.interpolate()

    for i, img in enumerate(images):
        img.calc = make_gpaw(f"Li-step-NEB-img{i}")

    if world.rank == 0:
        print(f"Starting NEB with {len(images)} images (target fmax = {neb_fmax:.3f} eV/Å)")

    neb_traj_path = os.path.join(output_dir, "Li-step-NEB.traj")
    opt = FIRE(
        neb,
        logfile=os.path.join(output_dir, "Li-step-NEB.log"),
        trajectory=neb_traj_path,  # save trajectory for restart
    )

    step = 0
    t0 = time.time()
    for _ in opt.irun(fmax=neb_fmax, steps=neb_max_steps):
        step += 1
        if world.rank == 0:
            elapsed = time.time() - t0
            try:
                neb_forces = neb.get_forces()
                maxf = float(np.abs(neb_forces).max())
            except Exception:
                maxf = float("nan")
            print(f"[Li-step NEB] step {step:4d}, NEB max |F| = {maxf:.3f} eV/Å, elapsed = {elapsed:7.1f} s")

    if world.rank == 0:
        total = time.time() - t0
        print(f"[Li-step NEB] optimization finished after {step} steps, total time = {total:.1f} s.")

    energies = np.array([img.get_potential_energy() for img in images])
    ref = min(energies[0], energies[-1])
    barrier = energies.max() - ref

    if world.rank == 0:
        print(f"Li step-edge diffusion barrier ≈ {barrier:.3f} eV")
        with open(os.path.join(output_dir, "Li-step-NEB-energies.dat"), "w") as f:
            f.write("# image   energy_eV\n")
            for i, E in enumerate(energies):
                f.write(f"{i:3d}  {E:15.8f}\n")
        print("Energies written to Li-step-NEB-energies.dat")

    # Mark NEB as done
    status["neb_done"] = True
    save_status(status)
    if world.rank == 0:
        print("NEB complete. Status saved.")

Starting NEB with 7 images (target fmax = 0.100 eV/Å)
[Li-step NEB] step    1, NEB max |F| = 0.215 eV/Å, elapsed = 10026.3 s
[Li-step NEB] step    2, NEB max |F| = 0.209 eV/Å, elapsed = 12032.5 s
[Li-step NEB] step    3, NEB max |F| = 0.194 eV/Å, elapsed = 13775.9 s
[Li-step NEB] step    4, NEB max |F| = 0.172 eV/Å, elapsed = 15857.4 s
[Li-step NEB] step    5, NEB max |F| = 0.146 eV/Å, elapsed = 18097.0 s
[Li-step NEB] step    6, NEB max |F| = 0.116 eV/Å, elapsed = 20256.1 s
[Li-step NEB] step    7, NEB max |F| = 0.084 eV/Å, elapsed = 22675.9 s
[Li-step NEB] optimization finished after 7 steps, total time = 22676.0 s.
Li step-edge diffusion barrier ≈ 0.051 eV
Energies written to Li-step-NEB-energies.dat
NEB complete. Status saved.


## 7. Visualization with NGLView

Visualize the initial/final structures and NEB trajectory animation using nglview.

In [13]:
import nglview as nv
from ase.io import read, Trajectory, write
import ipywidgets as widgets
from IPython.display import display
import tempfile

def show_structure(atoms, title="Structure", size=(600, 600)):
    """Display a single ASE Atoms object with nglview."""
    view = nv.show_ase(atoms)
    view._set_size(f"{size[0]}px", f"{size[1]}px")
    view.clear_representations()
    view.add_representation('spacefill', radius=0.5)
    view.add_representation('unitcell')
    view.parameters = dict(backgroundColor='white')
    view.camera = 'orthographic'
    print(title)
    return view

def load_neb_images_from_xyz(output_dir, n_images=7):
    """Load final NEB images from individual XYZ files if available,
    otherwise reconstruct from initial/final relaxed structures."""
    images = []
    
    # Try loading from initial and final relaxed structures
    initial_path = os.path.join(output_dir, "Li-initial-relaxed.xyz")
    final_path = os.path.join(output_dir, "Li-final-relaxed.xyz")
    
    if os.path.exists(initial_path) and os.path.exists(final_path):
        initial = read(initial_path)
        final = read(final_path)
        
        # Interpolate positions between initial and final
        pos_init = initial.get_positions()
        pos_final = final.get_positions()
        
        for i in range(n_images):
            t = i / (n_images - 1)  # interpolation parameter 0 to 1
            atoms = initial.copy()
            pos_interp = (1 - t) * pos_init + t * pos_final
            atoms.set_positions(pos_interp)
            images.append(atoms)
        
        return images
    
    return None

def show_neb_animation(output_dir, title="NEB Animation", size=(800, 800), n_images=7):
    """Display NEB path as an animation using interpolated structures."""
    
    # Load or interpolate NEB images
    images = load_neb_images_from_xyz(output_dir, n_images)
    
    if images is None or len(images) == 0:
        print("Could not load NEB images")
        return None
    
    print(f"Loaded {len(images)} NEB images for animation")
    
    # Create nglview trajectory widget
    view = nv.show_asetraj(images)
    view._set_size(f"{size[0]}px", f"{size[1]}px")
    view.clear_representations()
    view.add_representation('spacefill', radius=0.5)
    view.add_representation('unitcell')
    view.parameters = dict(backgroundColor='white')
    view.camera = 'orthographic'
    
    print(f"{title} ({len(images)} images)")
    print("Use the frame slider or player controls to view the diffusion path")
    return view

def show_initial_final_comparison(initial_path, final_path, title_prefix=""):
    """Display initial and final structures side by side."""
    initial = read(initial_path)
    final = read(final_path)
    
    print(f"{title_prefix} Initial vs Final Structures")
    print("-" * 40)
    
    # Initial structure
    view1 = show_structure(initial, f"{title_prefix} Initial", size=(500, 500))
    display(view1)
    
    # Final structure
    view2 = show_structure(final, f"{title_prefix} Final", size=(500, 500))
    display(view2)
    
    return view1, view2


print("Visualization functions defined.")

Visualization functions defined.


### 7.1 Visualize Initial and Final structures

In [5]:
# Visualize initial and final structures for Li step-edge diffusion
initial_path = os.path.join(output_dir, "Li-initial-relaxed.xyz")
final_path = os.path.join(output_dir, "Li-final-relaxed.xyz")

if os.path.exists(initial_path) and os.path.exists(final_path):
    print("="*60)
    print("Li(110) Step-edge - Initial Structure (Lower Terrace)")
    print("="*60)
    initial_atoms = read(initial_path)
    view_init = show_structure(initial_atoms, "Initial (Adatom on Lower Terrace)", size=(600, 600))
    display(view_init)
    
    print("\nLi(110) Step-edge - Final Structure (Upper Terrace)")
    final_atoms = read(final_path)
    view_final = show_structure(final_atoms, "Final (Adatom on Upper Terrace)", size=(600, 600))
    display(view_final)
else:
    print("Structure files not found. Run the relaxation cells first.")

Li(110) Step-edge - Initial Structure (Lower Terrace)
Initial (Adatom on Lower Terrace)


NGLWidget()


Li(110) Step-edge - Final Structure (Upper Terrace)
Final (Adatom on Upper Terrace)


NGLWidget()

### 7.2 NEB Trajectory Animation

Animate the step-edge diffusion path showing all NEB images.

In [14]:
# Animate NEB trajectory for Li step-edge diffusion
print("="*60)
print("Li(110) Step-edge - NEB Diffusion Animation")
print("="*60)
print("Animation shows Li adatom diffusing from lower to upper terrace")
print()

# Number of NEB images (initial + n_middle_images + final)
total_neb_images = n_middle_images + 2  # Should be 7

view = show_neb_animation(output_dir, "Li Step-edge NEB Trajectory", 
                           size=(800, 800), n_images=total_neb_images)
if view:
    display(view)
    
    # Create a manual frame slider for better control
    frame_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=total_neb_images - 1,
        step=1,
        description='Frame:',
        continuous_update=True
    )
    
    def on_frame_change(change):
        view.frame = change['new']
    
    frame_slider.observe(on_frame_change, names='value')
    
    # Play/Pause button
    play_button = widgets.Play(
        value=0,
        min=0,
        max=total_neb_images - 1,
        step=1,
        interval=800,  # ms between frames
        description="Play"
    )
    widgets.jslink((play_button, 'value'), (frame_slider, 'value'))
    
    print("\nUse controls below to animate:")
    display(widgets.HBox([play_button, frame_slider]))

Li(110) Step-edge - NEB Diffusion Animation
Animation shows Li adatom diffusing from lower to upper terrace

Loaded 7 NEB images for animation
Li Step-edge NEB Trajectory (7 images)
Use the frame slider or player controls to view the diffusion path


NGLWidget(max_frame=6)


Use controls below to animate:


### 7.3 Stepped Slab Visualization

View the stepped Li(110) slab structure.

In [7]:
# Visualize the stepped Li(110) slab
slab_path = os.path.join(output_dir, "Li-step-slab.xyz")

if os.path.exists(slab_path):
    print("="*60)
    print("Li(110) Stepped Slab Structure")
    print("="*60)
    print("The step edge separates lower and upper terraces")
    slab_atoms = read(slab_path)
    view_slab = show_structure(slab_atoms, "Li(110) Stepped Slab", size=(700, 700))
    display(view_slab)
else:
    print("Slab structure file not found. Run the slab building cell first.")

Li(110) Stepped Slab Structure
The step edge separates lower and upper terraces
Li(110) Stepped Slab


NGLWidget()